<a href="https://colab.research.google.com/github/VenkateshDas/legal_text_retrieval_coliee/blob/master/classificationTask_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Steps involved

Dataset creation : 

1.   Read the pickle files. "Cleaned Civil code" and "Cleaned Ground truth" 
2.   Create a dataset with columns : 
    ID , Query , Article , Label
3. Should there be any preprocess on the Dataset? 
4. Run a loop to concatenate query with every article. If the Relevant article number matches the article number then label it as "1" else "0". 
5. Then extract only the instances with label 1. 
6. Duplicate the rows of the dataset 100 times. 
7. Concatenate the duplicated or oversampled dataset with the dataset that contains the label 0 instances. 
8. Concatenate the Query and Article into a single stirng which will be fed as an input to the Transformer model. (Should I use the separate [SEP] token between Query and Article?)





### Setup 

In [ ]:
import os
import re
import ast
import sys
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
os.chdir('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/')

query_dataframe = pd.read_pickle('cleaned_ground_truth.pkl')
civil_code_dataframe = pd.read_pickle('cleaned_civil_code.pkl')


In [ ]:
query_dataframe.head()

,ID,Query,Article_numbers,Query_tokens,Query_lemma,Query_pos,Articles
0,H21-1-1,Acceptance made by a minor that received an o...,['5'],"['acceptance', 'made', 'minor', 'received', 'o...","['acceptance', 'make', 'minor', 'receive', 'of...","['NOUN', 'VERB', 'NOUN', 'VERB', 'NOUN', 'NOUN...",\nArticle 5\n(1) A minor must obtain the conse...
1,H21-1-2,"If a minor, after getting permission for carr...",['6'],"['minor', 'getting', 'permission', 'carrying',...","['minor', 'get', 'permission', 'carry', 'busin...","['ADJ', 'VERB', 'NOUN', 'VERB', 'NOUN', 'ADJ',...",\nArticle 6\n(1) A minor who is permitted to c...
2,H21-1-3,The purchase of daily household items may not...,['9'],"['purchase', 'daily', 'household', 'items', 'm...","['purchase', 'daily', 'household', 'item', 'ma...","['NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADV',...",\nArticle 9\nA juridical act performed by an a...
3,H21-1-4,"If a person under curatorship performs, witho...","['13', '120', '122']","['person', 'curatorship', 'performs', 'without...","['person', 'curatorship', 'perform', 'without'...","['NOUN', 'NOUN', 'NOUN', 'ADP', 'VERB', 'NOUN'...",\nArticle 13\n(1) A person under curatorship m...
4,H21-1-5,The person in question may not request a com...,"['7', '11']","['person', 'question', 'may', 'not', 'request'...","['person', 'question', 'may', 'not', 'request'...","['NOUN', 'NOUN', 'VERB', 'ADV', 'VERB', 'NOUN'...",\nArticle 7\nThe family court may decide to co...


In [ ]:
civil_code_dataframe.tail()

,Article_number,Referenced_article_num,Article_description,Article_titles,Article_description_tokens,Article_description_lemmas,Article_description_pos,Article_desc_bigrams,Article_desc_trigrams,Article_bi_tri_grams,Combined_lemma_Bi_tri,Article_title_tokens,Article_title_lemma,Article_title_pos
772,721,{},(Fetus' Capacity to Hold Rights Regarding Clai...,(Fetus' Capacity to Hold Rights Regarding Clai...,"[fetus, capacity, hold, rights, regarding, cla...","[fetus, capacity, hold, right, regard, claim, ...","[X, NOUN, VERB, NOUN, VERB, NOUN, NOUN, NOUN, ...","[fetus_capacity, capacity_hold, hold_right, ri...","[fetus_capacity_hold, capacity_hold_right, hol...","[fetus_capacity, capacity_hold, hold_right, ri...","[fetus, capacity, hold, right, regard, claim, ...","[fetus, capacity, hold, rights, regarding, cla...","[fetus, capacity, hold, right, regard, claim, ...","[X, NOUN, VERB, NOUN, VERB, NOUN, NOUN, NOUN, ..."
773,722,{},"(Method of Compensation for Loss or Damage, De...","(Method of Compensation for Loss or Damage, De...","[method, compensation, loss, damage, deduction...","[method, compensation, loss, damage, deduction...","[NOUN, NOUN, NOUN, NOUN, NOUN, ADJ, NOUN, ADJ,...","[method_compensation, compensation_loss, loss_...","[method_compensation_loss, compensation_loss_d...","[method_compensation, compensation_loss, loss_...","[method, compensation, loss, damage, deduction...","[method, compensation, loss, damage, deduction...","[method, compensation, loss, damage, deduction...","[NOUN, NOUN, NOUN, NOUN, NOUN, ADJ, NOUN, ADJ,..."
774,723,{},(Recovery in Defamation) Article_723 The cour...,(Recovery in Defamation),"[recovery, defamation, article_723, court, may...","[recovery, defamation, article_723, court, may...","[NOUN, NOUN, X, NOUN, VERB, VERB, NOUN, VERB, ...","[recovery_defamation, defamation_article_723, ...","[recovery_defamation_article_723, defamation_a...","[recovery_defamation, defamation_article_723, ...","[recovery, defamation, article_723, court, may...","[recovery, defamation]","[recovery, defamation]","[NOUN, NOUN]"
775,724,{},(Extinctive Prescription of Claim for Compensa...,(Extinctive Prescription of Claim for Compensa...,"[extinctive, prescription, claim, compensation...","[extinctive, prescription, claim, compensation...","[ADJ, NOUN, NOUN, NOUN, NOUN, NOUN, VERB, NOUN...","[extinctive_prescription, prescription_claim, ...","[extinctive_prescription_claim, prescription_c...","[extinctive_prescription, prescription_claim, ...","[extinctive, prescription, claim, compensation...","[extinctive, prescription, claim, compensation...","[extinctive, prescription, claim, compensation...","[ADJ, NOUN, NOUN, NOUN, NOUN, NOUN, VERB, NOUN]"
776,724_2,{},(Extinctive Prescription of Claim for Compensa...,(Extinctive Prescription of Claim for Compensa...,"[extinctive, prescription, claim, compensation...","[extinctive, prescription, claim, compensation...","[ADJ, NOUN, NOUN, NOUN, NOUN, NOUN, VERB, NOUN...","[extinctive_prescription, prescription_claim, ...","[extinctive_prescription_claim, prescription_c...","[extinctive_prescription, prescription_claim, ...","[extinctive, prescription, claim, compensation...","[extinctive, prescription, claim, compensation...","[extinctive, prescription, claim, compensation...","[ADJ, NOUN, NOUN, NOUN, NOUN, NOUN, VERB, NOUN..."


In [ ]:
civil_code_dataframe.shape[0]

776

In [ ]:
query_dataframe.shape[0]

695

In [ ]:
query_dataframe.iloc[186]

ID                                                           H29-2-A
Query               The person under curatorship A sold A's land ...
Article_numbers                                               ['20']
Query_tokens       ['person', 'curatorship', 'sold', "'s", 'land'...
Query_lemma        ['person', 'curatorship', 'sell', "'s", 'land'...
Query_pos          ['NOUN', 'NOUN', 'VERB', 'PART', 'NOUN', 'PUNC...
Articles           \nArticle 20\n(1) After a person with qualifie...
Name: 186, dtype: object

### Classification Traning dataset

In [ ]:
basic_classification_data_dict = {}

serial_num = 0
serial_num_list = []
id_list = []
query_list = []
relevant_article_number_list = []
article_number_list = []
article_list = []

for query_item in range(query_dataframe.shape[0]):
    print(query_item)
    serial_num = serial_num + 1
    for civil_code in range(civil_code_dataframe.shape[0]):
        id = query_dataframe.iloc[query_item]['ID']
        query = query_dataframe.iloc[query_item]['Query']
        relevant_article_number = query_dataframe.iloc[query_item]['Article_numbers']
        article_number = civil_code_dataframe.iloc[civil_code]['Article_number']
        article = civil_code_dataframe.iloc[civil_code]['Article_description']

        serial_num_list.append(serial_num)
        id_list.append(id)
        query_list.append(query)
        relevant_article_number_list.append(relevant_article_number)
        article_number_list.append(article_number)
        article_list.append(article)

        
basic_classification_data_dict.update({'Serial_Num':serial_num_list,'ID':id_list , 'Query': query_list , 'Relevant_article_number':relevant_article_number_list,'Article_number':article_number_list,'Article':article_list})
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# columns = ['ID','Query','Relevant_article_number','Article_number','Article']
basic_classification_dataframe = pd.DataFrame.from_dict(basic_classification_data_dict,orient = 'columns')

In [ ]:
basic_classification_dataframe.head()

,Serial_Num,ID,Query,Relevant_article_number,Article_number,Article
0,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],1,(Fundamental Principles) Article_1 (1) Privat...
1,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],2,(Standards for Construction) Article_2 This C...
2,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],3,Article 3 (1) The enjoyment of private rights...
3,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],3_2,Article 3-2 If the person making a juridical ...
4,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],4,(Age of Majority) Article_4 The age of majori...


In [ ]:
basic_classification_dataframe['Article_number'].iloc[0].split()[0]

'1'

#### Oversample training dataset (Relevant class)

In [ ]:
basic_classification_dataframe['label'] = 0
correct_labels = 0

for index in range(basic_classification_dataframe.shape[0]):
    if basic_classification_dataframe['Article_number'].iloc[index].split()[0] in ast.literal_eval(basic_classification_dataframe.iloc[index]['Relevant_article_number']):
        correct_labels = correct_labels + 1
        basic_classification_dataframe['label'].iloc[index] = 1

print(correct_labels)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


900


In [ ]:
#one preprocessing is the change of Reference Article to Ref_Article
basic_classification_dataframe['Query'].iloc[2341]+' '+basic_classification_dataframe['Article'].iloc[2341]

" If a person under curatorship performs, without getting the consent, an act that requires  getting consent from his/her curator, the curator may ratify that act, but may not rescind that act.  (Acts Requiring Consent of Curator) Article_13  (1) A person under curatorship must obtain the consent of the curator in order to perform any of the following acts; provided, however, that this does not apply to an act provided for in the proviso of Ref_Article : (i) receiving or using any property producing civil fruit; (ii) borrowing money or guaranteeing an obligation; (iii) performing an act with the purpose of acquiring or losing any right regarding immovables or other significant property; (iv) suing any procedural act; (v) giving a gift, reaching a settlement, or entering into an arbitration agreement (meaning an arbitration agreement as provided in Ref_Article , paragraph (1) of the Arbitration Act (Act No. 138 of 2003)); (vi) accepting or renouncing a succession or dividing an estate; 

In [ ]:
basic_classification_dataframe_cor = basic_classification_dataframe.loc[basic_classification_dataframe['label']==1]
basic_classification_dataframe_incor = basic_classification_dataframe.loc[basic_classification_dataframe['label']==0]

In [ ]:
basic_classification_dataframe_cor

,Serial_Num,ID,Query,Relevant_article_number,Article_number,Article,label
5,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],5,(Juridical Acts by Minors) Article_5 (1) A mi...,1
782,2,H21-1-2,"If a minor, after getting permission for carr...",['6'],6,(Permission for Minors to Conduct Business) Ar...,1
1561,3,H21-1-3,The purchase of daily household items may not...,['9'],9,(Juridical Acts by Adult Wards under Guardians...,1
2341,4,H21-1-4,"If a person under curatorship performs, witho...","['13', '120', '122']",13,(Acts Requiring Consent of Curator) Article_13...,1
2403,4,H21-1-4,"If a person under curatorship performs, witho...","['13', '120', '122']",120,(Holder of Right to Rescind) Article_120 (1) ...,1
...,...,...,...,...,...,...,...
536202,691,H24-29-1,A plaintiff shall prove that the opponent has...,['712'],712,(Capacity for Liability) Article_712 If a min...,1
536977,692,H24-29-3,Parents of a person who died by a tort may cl...,['711'],711,(Compensation for Loss or Damage to Close Rela...,1
537436,693,H24-30-1,In cases where an obligee A demands performan...,['452'],452,(Defense of Demand) Article_452 If an obligee...,1
538444,694,H24-30-2,In the cases where delivery of the subject ma...,['633'],633,(Timing of Payment of Remuneration) Article_63...,1


In [ ]:
basic_classification_dataframe_incor

,Serial_Num,ID,Query,Relevant_article_number,Article_number,Article,label
0,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],1,(Fundamental Principles) Article_1 (1) Privat...,0
1,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],2,(Standards for Construction) Article_2 This C...,0
2,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],3,Article 3 (1) The enjoyment of private rights...,0
3,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],3_2,Article 3-2 If the person making a juridical ...,0
4,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],4,(Age of Majority) Article_4 The age of majori...,0
...,...,...,...,...,...,...,...
539315,695,H24-30-3,In cases where A claims purchase money of mov...,['555'],721,(Fetus' Capacity to Hold Rights Regarding Clai...,0
539316,695,H24-30-3,In cases where A claims purchase money of mov...,['555'],722,"(Method of Compensation for Loss or Damage, De...",0
539317,695,H24-30-3,In cases where A claims purchase money of mov...,['555'],723,(Recovery in Defamation) Article_723 The cour...,0
539318,695,H24-30-3,In cases where A claims purchase money of mov...,['555'],724,(Extinctive Prescription of Claim for Compensa...,0


In [ ]:
def compute_hcf(x, y):

# choose the smaller number
    if x > y:
        smaller = y
    else:
        smaller = x
    for i in range(1, smaller+1):
        if((x % i == 0) and (y % i == 0)):
            hcf = i 
    return hcf




total = basic_classification_dataframe.shape[0]
incor = basic_classification_dataframe_incor.shape[0]
corr = basic_classification_dataframe_cor.shape[0]

hcf = compute_hcf(corr,incor)


corr_ratio = float(corr/hcf)
incorr_ratio = float(incor/hcf)

print(str(corr) + " : " + str(incor))
print(str(corr_ratio) + " : " + str(incorr_ratio))

corr_per = corr/total
incorr_per = incor/total



print(str(corr_per*100) + ","+ str(incorr_per*100))



900 : 538420
45.0 : 26921.0
0.16687680783208483,99.83312319216792


In [ ]:
#increasing by N times 

N = 595

ovr_corr = corr * N 

ovr_hcf = compute_hcf(corr,incor)

ovrcorr_ratio = float(ovr_corr/ovr_hcf)
ovrincorr_ratio = float(incor/ovr_hcf)

print(str(ovr_corr) + " : " + str(incor))
print(str(ovrcorr_ratio) + " : " + str(ovrincorr_ratio))

over_total = ovr_corr + incor 

ovrcorr_per = ovr_corr/over_total
ovrincorr_per = incor/over_total



print(str(ovrcorr_per*100) + ","+ str(ovrincorr_per*100))


535500 : 538420
26775.0 : 26921.0
49.8640494636472,50.1359505363528


In [ ]:
classification_data_dict = {}

combined_sequence_list = []
label_list = []


for index in range(basic_classification_dataframe.shape[0]):

    combined_sequence = basic_classification_dataframe['Query'].iloc[index]+' '+basic_classification_dataframe['Article'].iloc[index]
    label = basic_classification_dataframe['label'].iloc[index]

    combined_sequence_list.append(combined_sequence)
    label_list.append(label)

classification_data_dict.update({'Serial_Num':serial_num_list , 'ID':id_list , 'Sequence':combined_sequence_list,'Article_number':article_number_list,'Label':label_list})


In [ ]:
classification_dataframe = pd.DataFrame.from_dict(classification_data_dict,orient='columns')

In [ ]:
classification_dataframe

,Serial_Num,ID,Sequence,Article_number,Label
0,1,H21-1-1,Acceptance made by a minor that received an o...,1,0
1,1,H21-1-1,Acceptance made by a minor that received an o...,2,0
2,1,H21-1-1,Acceptance made by a minor that received an o...,3,0
3,1,H21-1-1,Acceptance made by a minor that received an o...,3_2,0
4,1,H21-1-1,Acceptance made by a minor that received an o...,4,0
...,...,...,...,...,...
539315,695,H24-30-3,In cases where A claims purchase money of mov...,721,0
539316,695,H24-30-3,In cases where A claims purchase money of mov...,722,0
539317,695,H24-30-3,In cases where A claims purchase money of mov...,723,0
539318,695,H24-30-3,In cases where A claims purchase money of mov...,724,0


In [ ]:
classification_dataframe.to_csv('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/classification_dataset.csv')

In [ ]:
classification_dataframe_cor = classification_dataframe.loc[basic_classification_dataframe['label']==1]
classification_dataframe_incor = classification_dataframe.loc[basic_classification_dataframe['label']==0]

In [ ]:
classification_dataframe_cor.head()

,ID,Sequence,Article_number,Label
5,H21-1-1,Acceptance made by a minor that received an o...,5,1
782,H21-1-2,"If a minor, after getting permission for carr...",6,1
1561,H21-1-3,The purchase of daily household items may not...,9,1
2341,H21-1-4,"If a person under curatorship performs, witho...",13,1
2403,H21-1-4,"If a person under curatorship performs, witho...",120,1


In [ ]:
basic_classification_dataframe.head()

,Serial_Num,ID,Query,Relevant_article_number,Article_number,Article,label
0,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],1,(Fundamental Principles) Article_1 (1) Privat...,0
1,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],2,(Standards for Construction) Article_2 This C...,0
2,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],3,Article 3 (1) The enjoyment of private rights...,0
3,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],3_2,Article 3-2 If the person making a juridical ...,0
4,1,H21-1-1,Acceptance made by a minor that received an o...,['5'],4,(Age of Majority) Article_4 The age of majori...,0


In [ ]:
basic_classification_dataframe.iloc[57773]

Serial_Num                                                                75
ID                                                                  H19-13-O
Query                       In addition to the fixed principal, revolving...
Relevant_article_number                                            ['398_3']
Article_number                                                         398_3
Article                    (Scope of Secured Claims under Revolving Mortg...
label                                                                      1
Name: 57773, dtype: object

In [ ]:
count = 0
for i in basic_classification_dataframe['label']:
    if i == 1:
        count += 1
print(count)

900


In [ ]:
import time
cn = 0
for num in range(query_dataframe.shape[0]):
    cn = cn + len(ast.literal_eval(query_dataframe['Article_numbers'].iloc[num])) 

print(cn)

901


In [ ]:
''' Oversampling the Relevant class in the dataframe '''
N_times = 595
oversampled_dataframe = pd.concat([classification_dataframe_cor]*N_times, ignore_index=True)

In [ ]:
oversampled_dataframe.shape

(535500, 4)

In [ ]:
''' Concatenate the Oversampled Relevant class and the Irrelevant class '''
oversampled_classification_dataframe = pd.concat([oversampled_dataframe,classification_dataframe_incor], ignore_index=True)

In [ ]:
oversampled_classification_dataframe.shape

(1073920, 4)

In [ ]:
oversampled_classification_dataframe

,ID,Sequence,Article_number,Label
0,H21-1-1,Acceptance made by a minor that received an o...,5,1
1,H21-1-2,"If a minor, after getting permission for carr...",6,1
2,H21-1-3,The purchase of daily household items may not...,9,1
3,H21-1-4,"If a person under curatorship performs, witho...",13,1
4,H21-1-4,"If a person under curatorship performs, witho...",120,1
...,...,...,...,...
1073915,H24-30-3,In cases where A claims purchase money of mov...,721,0
1073916,H24-30-3,In cases where A claims purchase money of mov...,722,0
1073917,H24-30-3,In cases where A claims purchase money of mov...,723,0
1073918,H24-30-3,In cases where A claims purchase money of mov...,724,0


#### Save the oversampled classification training dataset 

In [ ]:
oversampled_classification_dataframe.to_csv('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/595_times_oversampled_classification_dataset.csv')
basic_classification_dataframe.to_csv('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/basic_classification_dataset.csv')

In [ ]:
oversampled_classification_dataframe.to_pickle('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/595_times_oversampled_classification_dataset.pkl')
basic_classification_dataframe.to_pickle('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/basic_classification_dataset.pkl')

### Classification Test Dataset

In [ ]:
df_test_data = pd.read_pickle('test_data/cleaned_formal_test_data.pkl')

In [ ]:
df_test_data.head()

,ID,Query,Query_tokens,Query_lemma,Query_pos,Expanded_query_tokens,Expanded_query_lemma,Query_bigrams,Query_trigrams,Query_bi_tri_grams,Expanded_query_bi_tri_grams
0,R01-1-A,A contract of sales concluded by a minor may ...,"[contract, sales, concluded, minor, may, not, ...","[contract, sale, conclude, minor, may, not, re...","[NOUN, NOUN, VERB, ADJ, VERB, ADV, VERB, VERB,...","[refer, nipper, not, day_by_day, narrow, resci...","[refer, nipper, not, day_by_day, narrow, young...","[contract_sale, sale_conclude, conclude_minor,...","[contract_sale_conclude, sale_conclude_minor, ...","[contract_sale, sale_conclude, conclude_minor,...","[refer, nipper, not, day_by_day, narrow, young..."
1,R01-1-I,A contract of sales concluded by an adult war...,"[contract, sales, concluded, adult, ward, may,...","[contract, sale, conclude, adult, ward, may, r...","[NOUN, NOUN, VERB, NOUN, NOUN, VERB, VERB, ADP...","[refer, day_by_day, narrow, rescinded, day-to-...","[refer, day_by_day, narrow, day-to-day, living...","[contract_sale, sale_conclude, conclude_adult,...","[contract_sale_conclude, sale_conclude_adult, ...","[contract_sale, sale_conclude, conclude_adult,...","[refer, day_by_day, narrow, day-to-day, living..."
2,R01-1-U,A contract of guarantee concluded by a person...,"[contract, guarantee, concluded, person, curat...","[contract, guarantee, conclude, person, curato...","[NOUN, NOUN, VERB, NOUN, NOUN, VERB, ADV, VERB...","[somebody, not, narrow, rescinded, reason, con...","[somebody, not, narrow, reason, consent, concl...","[contract_guarantee, guarantee_conclude, concl...","[contract_guarantee_conclude, guarantee_conclu...","[contract_guarantee, guarantee_conclude, concl...","[somebody, not, narrow, reason, consent, concl..."
3,R01-1-E,An act which requires the consent of the assi...,"[act, requires, consent, assistant, may, resci...","[act, require, consent, assistant, may, rescin...","[NOUN, VERB, NOUN, NOUN, VERB, VERB, NOUN, NOU...","[fellowship, stead, somebody, folk, household,...","[fellowship, stead, somebody, folk, household,...","[act_require, require_consent, consent_assista...","[act_require_consent, require_consent_assistan...","[act_require, require_consent, consent_assista...","[fellowship, stead, somebody, folk, household,..."
4,R01-1-O,The period of the extinctive prescription of ...,"[period, extinctive, prescription, right, resc...","[period, extinctive, prescription, right, resc...","[NOUN, ADJ, NOUN, NOUN, VERB, NOUN, VERB, NOUN...","[extinctive, turn, capability, geological_peri...","[extinctive, turn, capability, geological_peri...","[period_extinctive, extinctive_prescription, p...","[period_extinctive_prescription, extinctive_pr...","[period_extinctive, extinctive_prescription, p...","[extinctive, turn, capability, geological_peri..."


#### Test data set creation 

In [ ]:
def parse_trec_rel_txt_file(qrel_file_path):
    text_file = open(qrel_file_path,'r')
    rel_text = text_file.readlines()

    text_list = []
    for item in rel_text:
        text = item.split()
        text_list.append(text)

    id_list = []
    article_list = []

    for i in text_list:
        id_list.append(i[0])
        article_list.append(i[2])

    rel_text_dict = {}
    rel_article_list = []
    set_id = id_list

    rel_article_dict = {}
    for i in range(len(id_list)):
        rel_list = []
        for j in range(len(id_list)):
            if id_list[i] == id_list[j]:
                rel_list.append(article_list[j])
        rel_article_dict.update({id_list[i]:rel_list})

    return rel_article_dict

In [ ]:
qrel_file_path = "/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /Formal_Run_Test_Data/qrel_task3.txt"

rel_article_dict = parse_trec_rel_txt_file(qrel_file_path)

In [ ]:
test_file_path = "/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /Formal_Run_Test_Data/"

In [ ]:
civil_code_dataframe.head()

,Article_number,Referenced_article_num,Article_description,Article_titles,Article_description_tokens,Article_description_lemmas,Article_description_pos,Article_desc_bigrams,Article_desc_trigrams,Article_bi_tri_grams,Combined_lemma_Bi_tri,Article_title_tokens,Article_title_lemma,Article_title_pos
1,1,{},(Fundamental Principles) Article_1 (1) Privat...,(Fundamental Principles),"[fundamental, principles, article_1, 1, privat...","[fundamental, principle, article_1, 1, private...","[ADJ, NOUN, NOUN, NUM, ADJ, NOUN, VERB, ADJ, A...","[fundamental_principle, principle_article_1, a...","[fundamental_principle_article_1, principle_ar...","[fundamental_principle, principle_article_1, a...","[fundamental, principle, article_1, 1, private...","[fundamental, principles]","[fundamental, principle]","[ADJ, NOUN]"
2,2,{},(Standards for Construction) Article_2 This C...,(Standards for Construction),"[standards, construction, article_2, code, mus...","[standard, construction, article_2, code, must...","[NOUN, NOUN, ADP, NOUN, VERB, VERB, VERB, NOUN...","[standard_construction, construction_article_2...","[standard_construction_article_2, construction...","[standard_construction, construction_article_2...","[standard, construction, article_2, code, must...","[standards, construction]","[standard, construction]","[NOUN, NOUN]"
3,3,{},Article 3 (1) The enjoyment of private rights...,(Standards for Construction),"[article, 3, 1, enjoyment, private, rights, co...","[article, 3, 1, enjoyment, private, right, com...","[NOUN, NUM, PUNCT, NOUN, ADJ, NOUN, NOUN, NOUN...","[article_3, 3_1, 1_enjoyment, enjoyment_privat...","[article_3_1, 3_1_enjoyment, 1_enjoyment_priva...","[article_3, 3_1, 1_enjoyment, enjoyment_privat...","[article, 3, 1, enjoyment, private, right, com...","[standards, construction]","[standard, construction]","[NOUN, NOUN]"
4,3_2,{},Article 3-2 If the person making a juridical ...,(Standards for Construction),"[article, 3, 2, person, making, juridical, act...","[article, 3, 2, person, make, juridical, act, ...","[NOUN, NUM, NUM, NOUN, VERB, ADJ, NOUN, ADV, A...","[article_3, 3_2, 2_person, person_make, make_j...","[article_3_2, 3_2_person, 2_person_make, perso...","[article_3, 3_2, 2_person, person_make, make_j...","[article, 3, 2, person, make, juridical, act, ...","[standards, construction]","[standard, construction]","[NOUN, NOUN]"
5,4,{},(Age of Majority) Article_4 The age of majori...,(Age of Majority),"[age, majority, article_4, age, majority, 20, ...","[age, majority, article_4, age, majority, 20, ...","[NOUN, NOUN, NOUN, NOUN, NOUN, NUM, NOUN, NOUN]","[age_majority, majority_article_4, article_4_a...","[age_majority_article_4, majority_article_4_ag...","[age_majority, majority_article_4, article_4_a...","[age, majority, article_4, age, majority, 20, ...","[age, majority]","[age, majority]","[NOUN, NOUN]"


In [ ]:
def parse_test_xml(file_path): 

    PairID_arr= []
    text2_arr = []
   
    count = 0
    sum = 0

    for files in os.listdir(file_path):
        try: 

            if files.endswith('.xml'):
                print(files)
                count += 1
                xmlp = ET.XMLParser(encoding="utf-8")
                tree = ET.parse(file_path+files, parser = xmlp)
                root = tree.getroot()
                count_list = []
                for pair in root.findall('pair'):
                    
                    PairID = pair.get('id')
                    count_list.append(PairID)
                    PairID_arr.append(PairID)
              
                    text2 = pair.find('t2').text
                    text2_arr.append(str(text2))

                sum += len(count_list)
                # print(len(count_list))
                # print(count_list)
        except Exception as e: 
            print(e , files)
    # print(count)
    print(sum)
    return PairID_arr,  text2_arr 

In [ ]:
def create_test_dataframe(PairID_arr,text2_arr):    
    dict_relevant = {'ID' : PairID_arr , 'Query' : text2_arr }
    df_test = pd.DataFrame(dict_relevant)
    return df_test

In [ ]:
def remove_newline_test_data(test_dataframe):

    new_line_pattern = re.compile(r'\n')
    for i in range(test_dataframe.shape[0]):
        test_dataframe['Query'].iloc[i] = re.sub(new_line_pattern,'',test_dataframe['Query'].iloc[i])
    return test_dataframe

In [ ]:
PairID_arr,  text2_arr  = parse_test_xml(test_file_path)

TestData_en-2.xml
112


In [ ]:
test_dataframe = create_test_dataframe(PairID_arr,text2_arr)
test_dataframe.head()

,ID,Query
0,R01-1-A,\nA contract of sales concluded by a minor may...
1,R01-1-I,\nA contract of sales concluded by an adult wa...
2,R01-1-U,\nA contract of guarantee concluded by a perso...
3,R01-1-E,\nAn act which requires the consent of the ass...
4,R01-1-O,\nThe period of the extinctive prescription of...


In [ ]:
test_dataframe = remove_newline_test_data(test_dataframe)
test_dataframe.head()

,ID,Query
0,R01-1-A,A contract of sales concluded by a minor may n...
1,R01-1-I,A contract of sales concluded by an adult ward...
2,R01-1-U,A contract of guarantee concluded by a person ...
3,R01-1-E,An act which requires the consent of the assis...
4,R01-1-O,The period of the extinctive prescription of t...


In [ ]:
#test classification data set 
basic_classification_test_data_dict = {}

test_serial_num = 0
test_serial_num_list = []
test_id_list = []
test_query_list = []
civil_article_number_list = []
civil_article_list = []
rel_article_num_list = []

for test_query_item in range(test_dataframe.shape[0]):
    print(test_query_item)
    test_serial_num = test_serial_num + 1
    for civil_code in range(civil_code_dataframe.shape[0]):
        test_id = test_dataframe.iloc[test_query_item]['ID']
        test_query = query_dataframe.iloc[test_query_item]['Query']
        rel_article_num = rel_article_dict[test_id]
        civil_article_number = civil_code_dataframe.iloc[civil_code]['Article_number']
        civil_article = civil_code_dataframe.iloc[civil_code]['Article_description']

        test_serial_num_list.append(test_serial_num)
        test_id_list.append(test_id)
        test_query_list.append(test_query)
        rel_article_num_list.append(rel_article_num)
        civil_article_number_list.append(civil_article_number)
        civil_article_list.append(civil_article)

        
basic_classification_test_data_dict.update({'Serial_Num':test_serial_num_list,'ID':test_id_list , 'Query': test_query_list ,'Relevant_Article_num':rel_article_num_list ,'Article_number':civil_article_number_list,'Article':civil_article_list})
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111


In [ ]:
basic_classification_test_dataframe = pd.DataFrame.from_dict(basic_classification_test_data_dict,orient = 'columns')

In [ ]:
basic_classification_test_dataframe.head()

,Serial_Num,ID,Query,Relevant_Article_num,Article_number,Article
0,1,R01-1-A,Acceptance made by a minor that received an o...,[5],1,(Fundamental Principles) Article_1 (1) Privat...
1,1,R01-1-A,Acceptance made by a minor that received an o...,[5],2,(Standards for Construction) Article_2 This C...
2,1,R01-1-A,Acceptance made by a minor that received an o...,[5],3,Article 3 (1) The enjoyment of private rights...
3,1,R01-1-A,Acceptance made by a minor that received an o...,[5],3_2,Article 3-2 If the person making a juridical ...
4,1,R01-1-A,Acceptance made by a minor that received an o...,[5],4,(Age of Majority) Article_4 The age of majori...


In [ ]:
basic_classification_test_dataframe.shape

(86912, 6)

In [ ]:
basic_classification_test_dataframe['Article_number'].iloc[0].split()[0]

'1'

In [ ]:
basic_classification_test_dataframe.iloc[0]['Relevant_Article_num'][0]

'5'

In [ ]:
basic_classification_test_dataframe['label'] = 0
correct_test_labels = 0

for index in range(basic_classification_test_dataframe.shape[0]):
    if basic_classification_test_dataframe['Article_number'].iloc[index].split()[0] in basic_classification_test_dataframe.iloc[index]['Relevant_Article_num']:
        correct_test_labels = correct_test_labels + 1
        basic_classification_test_dataframe['label'].iloc[index] = 1

print(correct_test_labels)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


138


In [ ]:
classification_test_data_dict = {}


combined_test_sequence_list = []
test_article_number_list = []
test_label_list = []

for index in range(basic_classification_test_dataframe.shape[0]):

    combined_test_sequence = basic_classification_test_dataframe['Query'].iloc[index]+' '+basic_classification_test_dataframe['Article'].iloc[index]
    test_article_number = basic_classification_test_dataframe['Article_number'].iloc[index]
    label = basic_classification_test_dataframe['label'].iloc[index]

    test_label_list.append(label)
    combined_test_sequence_list.append(combined_test_sequence)
    test_article_number_list.append(test_article_number)

classification_test_data_dict.update({'ID':test_id_list,'Sequence':combined_test_sequence_list,'Article_number':test_article_number_list, 'Label': test_label_list})


In [ ]:
classification_test_dataframe = pd.DataFrame.from_dict(classification_test_data_dict,orient = 'columns')

In [ ]:
classification_test_dataframe.head()

,ID,Sequence,Article_number,Label
0,R01-1-A,Acceptance made by a minor that received an o...,1,0
1,R01-1-A,Acceptance made by a minor that received an o...,2,0
2,R01-1-A,Acceptance made by a minor that received an o...,3,0
3,R01-1-A,Acceptance made by a minor that received an o...,3_2,0
4,R01-1-A,Acceptance made by a minor that received an o...,4,0


#### Save the classification test dataset

In [ ]:
basic_classification_test_dataframe.to_csv('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/basic_classification_test_dataset.csv')
classification_test_dataframe.to_csv('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/classification_test_dataset.csv')

In [ ]:
basic_classification_test_dataframe.to_pickle('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/basic_classification_test_dataset.pkl')
classification_test_dataframe.to_pickle('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/classification_test_dataset.pkl')